# LDA
### https://omicro03.medium.com/자연어처리-nlp-8일차-lda-f571b4da9d04
### https://towardsdatascience.com/topic-modeling-with-nlp-on-amazon-reviews-an-application-of-latent-dirichlet-allocation-lda-ae42a4c8b369

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
from sklearn.datasets import fetch_20newsgroups


/opt/anaconda3/envs/AI_dev/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choigww/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def get_news(stopwords):
    dataset = fetch_20newsgroups(shuffle=True,
                                random_state=42,
                                remove=('headers', 'footers', 'quotes'))
    documents = dataset.data
    news_df = pd.DataFrame({'document':documents})
    news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
    news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: " ".join([w for w in x.split()
                                                                              if len(w) > 2]))
                                                      
    news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
    tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())                                                
    return tokenized_doc.apply(lambda x: ' '.join([item for item in x if item not in stopwords]))

sw = stopwords.words('english')
tokenized_docs = get_news(sw)
tokenized_docs.head()

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


0    wondering anyone could enlighten car saw day d...
1    fair number brave souls upgraded clock oscilla...
2    well folks mac plus finally gave ghost weekend...
3    weitek address phone number like get informati...
4    article owcb world std com tombaker world std ...
Name: clean_doc, dtype: object

In [20]:
tokenized_docs.shape

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(11314,)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(tokenized_docs)
lda = LatentDirichletAllocation(n_components = 20)
lda_out = lda.fit_transform(tfidf)

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
print('done')

done


/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne') # 저차원 압축해서 시각화
pyLDAvis.display(vis)

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

lda_pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
                        ('lda', LatentDirichletAllocation(random_state=42))])

params = {'tfidf_vect__ngram_range':[(1, 1), (1, 2)],
          'lda__n_components': [5, 10, 20],
           'lda__max_iter': [10, 50, 100]
         }

gs_lda = GridSearchCV(lda_pipeline, params, n_jobs=-1)
gs_lda = gs_lda.fit(tokenized_docs)

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
print('done')

done


/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
print(gs_lda.best_params_)
print(gs_lda.best_score_)

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'lda__max_iter': 100,
 'lda__n_components': 5,
 'tfidf_vect__ngram_range': (1, 1)}

In [33]:
tfidf_vect = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vect.fit_transform(tokenized_docs)
vis = pyLDAvis.sklearn.prepare(gs_lda.best_estimator_.steps[1][1], tfidf, gs_lda.best_estimator_.steps[0][1], mds='tsne')
pyLDAvis.display(vis)

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
